In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from astropy.units import one
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import itertools
import random
import math
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, \
                            precision_recall_curve, average_precision_score, roc_auc_score, roc_curve, auc, accuracy_score

from IPython.display import clear_output

from tqdm import tqdm_notebook

import pandas as pd
# the distribution of sequence length
from matplotlib import pyplot as plt

In [2]:
# from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
# from torch.nn.utils.rnn import pad_sequence
# from torch import Tensor

# import torch
# import torch.nn.functional as F
# import torch.nn as nn
# import torch.utils.data as data_utils
# import torch.optim.lr_scheduler as lr_scheduler 
# import matplotlib.pyplot as plt
# import copy

In [3]:
import lightgbm as lgb
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score


import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt

# sklearn tools for model training and assesment
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score, balanced_accuracy_score, matthews_corrcoef)

In [4]:
np.random.seed(0)


## paths

In [9]:
project_dir = '../../../'

#ds_dir = f'{project_dir}data/seq/'
ds_dir = f'{project_dir}data/processed/'

snapshot_dir = f'{project_dir}snapshots/'

classes = ['lnc_e','lnc_p']
class_suffix = '_noN'

neg_filename = f'{ds_dir}train_val_test__80_20/{classes[0]}{class_suffix}_tr_val.csv'
pos_filename = f'{ds_dir}train_val_test__80_20/{classes[1]}{class_suffix}_tr_val.csv'

test_neg_filename = f'{ds_dir}train_val_test__80_20/{classes[0]}{class_suffix}_test.csv'
test_pos_filename = f'{ds_dir}train_val_test__80_20/{classes[1]}{class_suffix}_test.csv'


# feature_filenames_class_0 = [f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_1mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_2mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_3mer_features_tr_val.csv']
# feature_filenames_class_1 = [f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_1mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_2mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_3mer_features_tr_val.csv']


# test_feature_filenames_class_0 = [f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_1mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_2mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_3mer_features_test.csv']
# test_feature_filenames_class_1 = [f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_1mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_2mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_3mer_features_test.csv']


bio_feature_filenames_class_0 = [f'{ds_dir}train_val_test__80_20/{classes[0]}_nosim.fa.matrix_tr_val.csv']
bio_feature_filenames_class_1 = [f'{ds_dir}train_val_test__80_20/{classes[1]}_nosim.fa.matrix_tr_val.csv']


test_bio_feature_filenames_class_0 = [f'{ds_dir}train_val_test__80_20/{classes[0]}_nosim.fa.matrix_test.csv']
test_bio_feature_filenames_class_1 = [f'{ds_dir}train_val_test__80_20/{classes[1]}_nosim.fa.matrix_test.csv']



In [10]:
# load positive and negative sequence data to construct ground label list

with open(neg_filename,'r') as nf:
    neg_examples = [seq.strip() for seq in nf.readlines()]
    num_neg = len(neg_examples) # number of negative examples
    print(num_neg)

with open(pos_filename,'r') as pf:
    pos_examples = [seq.strip() for seq in pf.readlines()]
    num_pos = len(pos_examples) # number of positive examples
    print(num_pos)    

sequences_DS = neg_examples + pos_examples 
labels_DS = np.array([0]*num_neg + [1]*num_pos)   
len(sequences_DS), len(labels_DS)

scale_pos_weight = num_neg/num_pos

2312
5462


In [11]:
# load training + validation data

bio_features_class0 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in bio_feature_filenames_class_0], axis = 1)
bio_features_class1 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in bio_feature_filenames_class_1], axis = 1)

neg_bio_features = bio_features_class0
pos_bio_features = bio_features_class1

bio_features_DS = np.concatenate([bio_features_class0, bio_features_class1], axis = 0)    
print(neg_bio_features.shape, pos_bio_features.shape,bio_features_DS.shape)

(2312, 292) (5462, 292) (7774, 292)


In [12]:
# load test data

test_bio_features_class0 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in test_bio_feature_filenames_class_0], axis = 1)
test_bio_features_class1 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in test_bio_feature_filenames_class_1], axis = 1)

test_neg_bio_features = test_bio_features_class0
test_pos_bio_features = test_bio_features_class1

test_bio_features_DS = np.concatenate([test_bio_features_class0, test_bio_features_class1], axis = 0)    

test_labels_DS = np.array([0]*test_bio_features_class0.shape[0] + [1]*test_bio_features_class1.shape[0])  
print(test_neg_bio_features.shape, test_pos_bio_features.shape,test_bio_features_DS.shape, len(test_labels_DS))

(579, 292) (1366, 292) (1945, 292) 1945


In [13]:
def lgb_mcc_score(y_hat, data):
    y_true = data.get_label()
    y_pred = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'mcc', matthews_corrcoef(y_true, y_pred), True

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


def compute_sens_spec_mcc(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    #cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
    tn, fp, fn, tp = cm.ravel()

    specificity = tn/(fp+tn)
    sensitivity = tp/(tp+fn)
    
    return {'specificity':specificity, 'sensitivity': sensitivity, 'mcc':matthews_corrcoef(y_true, y_pred)}

In [14]:
# randomly shuffle the dataset (required as all negative class examples come first in the original)

indx = list(range(len(labels_DS)))
np.random.shuffle(indx)
labels_DS_np = np.array(labels_DS)
labels_shuf = labels_DS_np[indx]
bio_features_shuf = bio_features_DS[indx,:]


train_frac = 0.8
val_frac = 1 - train_frac

train_count = int(len(indx)*train_frac)
val_count = len(indx) - train_count

train_features = bio_features_shuf[:train_count,:]
train_labels = labels_shuf[:train_count]

val_features = bio_features_shuf[train_count:,:]
val_labels = labels_shuf[train_count:]

test_features = test_bio_features_DS
test_labels = test_labels_DS


train_dataset = lgb.Dataset(train_features, label=train_labels)
valid_dataset = lgb.Dataset(val_features, label=val_labels)
test_dataset = lgb.Dataset(test_features, label=test_labels)

train_val_dataset = lgb.Dataset(bio_features_DS, label=labels_DS_np)

print("train:", train_features.shape, len(train_labels))
print("val:", val_features.shape, len(val_labels))
print("test:", test_features.shape, len(test_labels))

print("train+val:", bio_features_DS.shape, len(labels_DS_np))

train: (6219, 292) 6219
val: (1555, 292) 1555
test: (1945, 292) 1945
train+val: (7774, 292) 7774


# Grid search to find the best parameters 

In [15]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'metric_freq': 1,
    'is_training_metric': True,
    'max_bin': 255,
    'learning_rate': 0.1,
    'num_leaves': 63,
    'tree_learner': 'serial',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 5,
    'is_enable_sparse': True,
    'use_two_round_loading': False,
    'is_save_binary_file': False,
    'output_model': 'LightGBM_model.txt',
    'num_machines': 1,
    'local_listen_port': 12400,
    'machine_list_file': 'mlist.txt',
    'verbose': 0,
    'subsample_for_bin': 200000,
    'min_child_samples': 20,
    'min_child_weight': 0.001,
    'min_split_gain': 0.0,
    'colsample_bytree': 1.0,
    'reg_alpha': 0.0,
    'reg_lambda': 0.0
}

gridParams = {
    'learning_rate': [ 0.05, 0.1, 0.2, 0.3],
    'num_leaves': [80, 100, 150, 200],
    'num_boost_round': [100, 150, 200],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary']
}

mdl = lgb.LGBMClassifier(
    task = params['task'],
    metric = 'matthews_corrcoef',
    metric_freq = params['metric_freq'],
    scale_pos_weight = scale_pos_weight,
    is_training_metric = params['is_training_metric'],
    max_bin = params['max_bin'],
    tree_learner = params['tree_learner'],
    feature_fraction = params['feature_fraction'],
    bagging_fraction = params['bagging_fraction'],
    bagging_freq = params['bagging_freq'],
    min_data_in_leaf = params['min_data_in_leaf'],
    min_sum_hessian_in_leaf = params['min_sum_hessian_in_leaf'],
    is_enable_sparse = params['is_enable_sparse'],
    use_two_round_loading = params['use_two_round_loading'],
    is_save_binary_file = params['is_save_binary_file'],
    n_jobs = -1
)

scoring = {'mcc': 'matthews_corrcoef'}

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=2, cv=10, n_jobs=-1, refit='mcc' )
# Run the grid
grid.fit(bio_features_DS, labels_DS )

print('Best parameters found by grid search are:', grid.best_params_)
print('Best score found by grid search is:', grid.best_score_)


Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.8s
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  2.2min finished
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Best parameters found by grid search are: {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'num_boost_round': 200, 'num_leaves': 80, 'objective': 'binary'}
Best score found by grid search is: 0.855544121430409


# K-fold cross validation using selected parameters  

In [16]:
# k-fold cross validation (built-in) on training + validation set (using selected params)
#num_round = 31
#param_nogrid = {'num_leaves': 100, 'num_boost_round': 100, 'objective': 'binary' }
# using best parameters from grid search result
param_nogrid = {'boosting_type': 'gbdt', 'learning_rate': 0.2, 'num_boost_round': 200, 'num_leaves': 80, 'objective': 'binary', 'scale_pos_weight':scale_pos_weight } 


param_nogrid['metric'] = 'mcc'

evals_result = {}

results = lgb.cv(param_nogrid, train_dataset, nfold= 10, feval = lgb_mcc_score, verbose_eval=0)
print(results['mcc-mean'][-1], results['mcc-stdv'][-1])

/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.6667882628766504 0.022779498712166772


/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.6802998642286617 0.03640292187559192


In [17]:
# k-fold cross validation on training + validation set (using selected params)

skf = StratifiedKFold( n_splits= 10, random_state = 23, shuffle=True)

results = []

models = []
for train_index, val_index in skf.split(bio_features_DS, labels_DS_np):
    train_dataset_kfold = lgb.Dataset(bio_features_DS[train_index,:], label=labels_DS_np[train_index])
    valid_dataset_kfold = lgb.Dataset(bio_features_DS[val_index,:], label=labels_DS_np[val_index])
    # train
    gbm = lgb.train(param_nogrid, train_dataset_kfold, feval = lgb_mcc_score, early_stopping_rounds = 5, valid_sets = valid_dataset_kfold, verbose_eval=0)
    
    models.append(gbm)
    results.append(compute_sens_spec_mcc(labels_DS_np[val_index], np.round(gbm.predict(bio_features_DS[val_index,:]))))
    

/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


# Results

# Cross validation results

In [18]:
specs = []
senss = []
mccs = []

for result in results:
    spec =  result['specificity']
    sens = result['sensitivity']
    mcc = result['mcc']

    specs.append(spec)
    senss.append(sens)
    mccs.append(mcc)
#print(matthews_corrcoef(labels_DS[indx[train_count:]], np.round(gbm.predict(bio_features_DS[indx[train_count:],:]))))  # validation data 
#print(matthews_corrcoef(labels_DS, gbm.predict(train_data)))


print("specificity:","mean:",np.mean(specs), "standard deviation:", np.std(specs) )
print("sensitivity:","mean:",np.mean(sens), "standard deviation:", np.std(sens) )
print("mcc:","mean:",np.mean(mccs), "standard deviation:", np.std(mccs) )

specificity: mean: 0.8382669055082849 standard deviation: 0.05960943143708073
sensitivity: mean: 0.8315018315018315 standard deviation: 0.0
mcc: mean: 0.6539740102929884 standard deviation: 0.021922523956461035


specificity: mean: 0.854675803124079 standard deviation: 0.02806946278720652
sensitivity: mean: 0.8452768729641694 standard deviation: 0.0
mcc: mean: 0.6565167252726619 standard deviation: 0.018990340559675023


In [21]:
best_model_idx = np.argmax(mccs)
gbm = models[best_model_idx]

# Test using the best model (with highest metric value)

In [22]:
compute_sens_spec_mcc(test_labels, np.round(gbm.predict(test_features)))

{'specificity': 0.8482758620689655,
 'sensitivity': 0.8243045387994143,
 'mcc': 0.6350657193454664}

# Test by retraining the model with best parameters, using the entire Train + Val set

In [23]:
gbm = lgb.train(param_nogrid, train_val_dataset, feval = lgb_mcc_score, verbose_eval=0)
compute_sens_spec_mcc(test_labels, np.round(gbm.predict(test_features)))

{'specificity': 0.803448275862069,
 'sensitivity': 0.8799414348462665,
 'mcc': 0.6680509143620029}

In [18]:
# # specify your configurations as a dict
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': {'binary_logloss', 'auc'},
#     'metric_freq': 1,
#     'is_training_metric': True,
#     'max_bin': 255,
#     'learning_rate': 0.1,
#     'num_leaves': 63,
#     'tree_learner': 'serial',
#     'feature_fraction': 0.8,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'min_data_in_leaf': 50,
#     'min_sum_hessian_in_leaf': 5,
#     'is_enable_sparse': True,
#     'use_two_round_loading': False,
#     'is_save_binary_file': False,
#     'output_model': 'LightGBM_model.txt',
#     'num_machines': 1,
#     'local_listen_port': 12400,
#     'machine_list_file': 'mlist.txt',
#     'verbose': 0,
#     'subsample_for_bin': 200000,
#     'min_child_samples': 20,
#     'min_child_weight': 0.001,
#     'min_split_gain': 0.0,
#     'colsample_bytree': 1.0,
#     'reg_alpha': 0.0,
#     'reg_lambda': 0.0
# }

# gridParams = {
#     'learning_rate': [ 0.1, 0.2, 0.3],
#     'num_leaves': [63, 80, 100],
#     'boosting_type' : ['gbdt'],
#     'objective' : ['binary']
# }

# mdl = lgb.LGBMClassifier(
#     task = params['task'],
#     metric = 'matthews_corrcoef',
#     metric_freq = params['metric_freq'],
#     is_training_metric = params['is_training_metric'],
#     max_bin = params['max_bin'],
#     tree_learner = params['tree_learner'],
#     feature_fraction = params['feature_fraction'],
#     bagging_fraction = params['bagging_fraction'],
#     bagging_freq = params['bagging_freq'],
#     min_data_in_leaf = params['min_data_in_leaf'],
#     min_sum_hessian_in_leaf = params['min_sum_hessian_in_leaf'],
#     is_enable_sparse = params['is_enable_sparse'],
#     use_two_round_loading = params['use_two_round_loading'],
#     is_save_binary_file = params['is_save_binary_file'],
#     n_jobs = -1
# )

# scoring = {'mcc': 'matthews_corrcoef'}

# # Create the grid
# grid = GridSearchCV(mdl, gridParams, verbose=2, cv=10, n_jobs=-1, refit='mcc' )
# # Run the grid
# grid.fit(bio_features_DS, labels_DS )

# print('Best parameters found by grid search are:', grid.best_params_)
# print('Best score found by grid search is:', grid.best_score_)

# # df_train = pd.read_csv("../../../data/sample/binary.train", header=None, sep='\t')
# # df_test = pd.read_csv("../../../data/sample/binary.test", header=None, sep='\t')

# # y_train = df_train[0].values
# # y_test = df_test[0].values
# # X_train = df_train.drop(0, axis=1).values
# # X_test = df_test.drop(0, axis=1).values

# # lgb_train = lgb.Dataset(X_train, y_train)
# # lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# # clf = lgb.train(param, train_data, valid_sets=[val_data, train_data], valid_names=['val', 'train'], feval=lgb_mcc_score, evals_result=evals_result)

# # lgb.plot_metric(evals_result, metric='f1')

In [19]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']